In [1]:
import os
from pyspark.sql import SparkSession

In [2]:
input_prefix = 'input'
# want test code to take SparkSession as input because we want this to be able to run on a spark cluster
spark = SparkSession.builder.getOrCreate()

In [3]:
from etl.load import load
from etl.clean import nullify_missing_values, parse_date

In [4]:
countrylist, stationlist, data = load(spark=spark, input_prefix=input_prefix)

In [5]:
data.show()

+------+-----+--------+----+----+------+------+-----+----+-----+-----+----+----+----+-----+------+
|STN---| WBAN|YEARMODA|TEMP|DEWP|   SLP|   STP|VISIB|WDSP|MXSPD| GUST| MAX| MIN|PRCP| SNDP|FRSHTT|
+------+-----+--------+----+----+------+------+-----+----+-----+-----+----+----+----+-----+------+
| 10260|99999|20190101|26.1|21.2|1001.9| 987.5| 20.6| 9.0| 15.9| 29.7|29.8|null|null| 18.5|001000|
| 10260|99999|20190102|24.9|22.1|1020.1|1005.5|  5.4| 5.6| 13.6| 22.1|null|20.7|null| 22.8|001000|
| 10260|99999|20190103|31.7|29.1|1008.9| 994.7| 13.6|11.6| 21.4| 49.5|null|null|null|999.9|011000|
| 10260|99999|20190104|32.9|30.3|1011.4| 997.1| 15.8| 4.9|  7.8| 10.9|36.1|31.8|null|999.9|001000|
| 10260|99999|20190105|35.5|33.0|1015.7|1001.4| 12.0|10.4| 13.6| 21.0|null|32.7|null| 23.6|010000|
| 10260|99999|20190106|38.5|34.1|1008.2| 994.2| 12.8|10.0| 17.5| 28.9|41.4|null|null| 23.2|010000|
| 10260|99999|20190107|32.1|29.8| 996.8| 982.7|  6.9|11.3| 15.5| 28.6|null|30.4|null|999.9|001000|
| 10260|99

In [6]:
data = nullify_missing_values(data)
data = parse_date(data)

In [7]:
data.show()

+------+-----+-------------------+----+----+------+------+-----+----+-----+----+----+----+----+----+------+
|STN---| WBAN|           YEARMODA|TEMP|DEWP|   SLP|   STP|VISIB|WDSP|MXSPD|GUST| MAX| MIN|PRCP|SNDP|FRSHTT|
+------+-----+-------------------+----+----+------+------+-----+----+-----+----+----+----+----+----+------+
| 10260|99999|2019-01-01 00:00:00|26.1|21.2|1001.9| 987.5| 20.6| 9.0| 15.9|29.7|29.8|null|null|18.5|001000|
| 10260|99999|2019-01-02 00:00:00|24.9|22.1|1020.1|1005.5|  5.4| 5.6| 13.6|22.1|null|20.7|null|22.8|001000|
| 10260|99999|2019-01-03 00:00:00|31.7|29.1|1008.9| 994.7| 13.6|11.6| 21.4|49.5|null|null|null|null|011000|
| 10260|99999|2019-01-04 00:00:00|32.9|30.3|1011.4| 997.1| 15.8| 4.9|  7.8|10.9|36.1|31.8|null|null|001000|
| 10260|99999|2019-01-05 00:00:00|35.5|33.0|1015.7|1001.4| 12.0|10.4| 13.6|21.0|null|32.7|null|23.6|010000|
| 10260|99999|2019-01-06 00:00:00|38.5|34.1|1008.2| 994.2| 12.8|10.0| 17.5|28.9|41.4|null|null|23.2|010000|
| 10260|99999|2019-01-07 00:

In [8]:
station_attr = countrylist.join(stationlist, on='COUNTRY_ABBR', how='outer')

In [9]:
data_full = data.join(
    station_attr.withColumnRenamed('STN_NO', 'STN---'),
    on='STN---',
    how='left'
).cache()

Step 2 - Questions

Using the global weather data, answer the following:

1. Which country had the hottest average mean temperature over the year?
2. Which country had the most consecutive days of tornadoes/funnel cloud
formations?
3. Which country had the second highest average mean wind speed over the year?

In [14]:
import pyspark.sql.functions as F

In [17]:
data_full.withColumn(
    'month', F.month(F.col('YEARMODA'))
).groupBy(
    'COUNTRY_FULL', 'month'
).agg(
    # possibly multiple weather stations (unverified)
    # thus aggregate by monthly first
    # ideally aggregate by day -> by month -> by year
    # leaving that for later if there's time
    F.mean('TEMP').alias('mean_monthly_TEMP')
).groupBy(
    'COUNTRY_FULL'
).agg(
    F.mean('mean_monthly_TEMP').alias('monthly_TEMP')
).orderBy(
    F.col('monthly_TEMP'), ascending=False
).show()

+--------------------+-----------------+
|        COUNTRY_FULL|     monthly_TEMP|
+--------------------+-----------------+
|            DJIBOUTI|90.23198015990424|
|                CHAD|87.14639472082757|
|               SUDAN|85.07542338470381|
|               NIGER|85.03855115744923|
| JUAN DE NOVA ISLAND|84.51182777650894|
|                MALI| 84.4698450533835|
|         EL SALVADOR|84.44803416216793|
|              TUVALU|84.34886658865116|
|BRITISH INDIAN OC...| 83.9194399820983|
|             TOKELAU|83.84275418635283|
|      CAYMAN ISLANDS|83.76236167953455|
|        BURKINA FASO|83.75583106326813|
|            MALDIVES|83.67978358497757|
|           SINGAPORE|83.63143542755746|
|ST. VINCENT AND T...|83.54223237327423|
|            CAMBODIA|83.50356529178372|
|    MARSHALL ISLANDS| 83.2364911782702|
|          MICRONESIA|83.23250023814408|
|             SENEGAL|83.13447681132101|
|            KIRIBATI|83.07528320407322|
+--------------------+-----------------+
only showing top

Q: Which country had the hottest average mean temperature over the year?

A: DJIBOUTI

makes, sense, it's in a dry part of Africa

Which country had the most consecutive days of tornadoes/funnel cloud formations?


In [44]:
from pyspark.sql.window import Window
from pyspark.sql.types import BooleanType
from pyspark.sql import DataFrame
from typing import List

from etl.pyspark_utils import consecutive_date_diff_by

# do we have daily observations for each country?
# can only compute for consecutive dates
country_date_diff = consecutive_date_diff_by(
    data_full,
    by_cols=['COUNTRY_FULL'],
    date_col='YEARMODA'
)

# filter for countries with inter_date only null or 1
country_single_consecutive = (
    country_date_diff
    .groupBy('COUNTRY_FULL')
    .agg(F.collect_set('inter_date').alias('set_inter_date'))
).withColumn(
    'is_single_consecutive',
    F.udf(lambda s: s == [1], BooleanType())(F.col('set_inter_date'))
)
country_single_consecutive.show()

+--------------------+--------------------+---------------------+
|        COUNTRY_FULL|      set_inter_date|is_single_consecutive|
+--------------------+--------------------+---------------------+
|             ARMENIA|                 [1]|                 true|
|        SOUTH AFRICA|                 [1]|                 true|
|               BURMA|                 [1]|                 true|
|            CAMBODIA|                 [1]|                 true|
|          BANGLADESH|                 [1]|                 true|
|               JAPAN|                 [1]|                 true|
|              UGANDA|        [1, 2, 3, 4]|                false|
|SOUTH GEORGIA AND...|                 [1]|                 true|
|          CAPE VERDE|              [1, 2]|                false|
|NORTHERN MARIANA ...|                 [1]|                 true|
|FALKLAND ISLANDS ...|                 [1]|                 true|
|          MAURITANIA|                 [1]|                 true|
|         

In [51]:
data_single_consecutive = data_full.join(
    country_single_consecutive.filter(F.col('is_single_consecutive')),
    on='COUNTRY_FULL',
    how='inner'
).withColumn(
    'has_tornado_or_funnel',
    F.udf(lambda s: s[5] == '1', BooleanType())('FRSHTT')
).select('COUNTRY_FULL', 'YEARMODA', 'has_tornado_or_funnel').distinct()
assert data_single_consecutive.count() > 0
assert data_single_consecutive.filter(F.col('has_tornado_or_funnel')).count() > 0

In [59]:
# https://stackoverflow.com/questions/54445961/pyspark-calculate-streak-of-consecutive-observations
# seems like i need a few windows ???

from pyspark.sql.window import Window

w_base = Window.partitionBy('COUNTRY_FULL').orderBy('YEARMODA')
w_flagged = Window.partitionBy('COUNTRY_FULL', 'has_tornado_or_funnel').orderBy('YEARMODA')

dftemp = data_single_consecutive.withColumn(
    'flag_group',
    # when 2 row counts increment, differences increase if there's a break
    F.row_number().over(w_base) - F.row_number().over(w_flagged)
)
dftemp.show()

# i don't really get the following part
w_streak = Window.partitionBy('COUNTRY_FULL', 'has_tornado_or_funnel', 'flag_group').orderBy('YEARMODA')
dftemp = (
    dftemp
    .withColumn(
        'streak_0',
        F.when(F.col('has_tornado_or_funnel'), 0).otherwise(F.row_number().over(w_streak))
    )
    .withColumn(
        'streak_1',
        F.when(~F.col('has_tornado_or_funnel'), 0).otherwise(F.row_number().over(w_streak))
    )
)
dftemp.show()

+------------+-------------------+---------------------+----------+
|COUNTRY_FULL|           YEARMODA|has_tornado_or_funnel|flag_group|
+------------+-------------------+---------------------+----------+
|     ARMENIA|2019-01-01 00:00:00|                false|         0|
|     ARMENIA|2019-01-02 00:00:00|                false|         0|
|     ARMENIA|2019-01-03 00:00:00|                false|         0|
|     ARMENIA|2019-01-04 00:00:00|                false|         0|
|     ARMENIA|2019-01-05 00:00:00|                false|         0|
|     ARMENIA|2019-01-06 00:00:00|                false|         0|
|     ARMENIA|2019-01-07 00:00:00|                false|         0|
|     ARMENIA|2019-01-08 00:00:00|                false|         0|
|     ARMENIA|2019-01-09 00:00:00|                false|         0|
|     ARMENIA|2019-01-10 00:00:00|                false|         0|
|     ARMENIA|2019-01-11 00:00:00|                false|         0|
|     ARMENIA|2019-01-12 00:00:00|              

In [61]:
dftemp.orderBy('streak_1', ascending=False).show()

+--------------+-------------------+---------------------+----------+--------+--------+
|  COUNTRY_FULL|           YEARMODA|has_tornado_or_funnel|flag_group|streak_0|streak_1|
+--------------+-------------------+---------------------+----------+--------+--------+
|        CANADA|2019-06-26 00:00:00|                 true|       176|       0|       2|
|         ITALY|2019-10-03 00:00:00|                 true|       275|       0|       2|
|         JAPAN|2019-04-02 00:00:00|                 true|        91|       0|       1|
|    BANGLADESH|2019-08-22 00:00:00|                 true|       233|       0|       1|
|         MALTA|2019-01-19 00:00:00|                 true|        18|       0|       1|
|         JAPAN|2019-12-04 00:00:00|                 true|       338|       0|       1|
|         JAPAN|2019-12-03 00:00:00|                 true|       336|       0|       1|
|        JERSEY|2019-06-12 00:00:00|                 true|       162|       0|       1|
|UNITED KINGDOM|2019-06-12 00:00

Q: Which country had the most consecutive days of tornadoes/funnel cloud formations?

A: Seems to be CANADA. Not sure.

----

In [18]:
data_full.withColumn(
    'month', F.month(F.col('YEARMODA'))
).groupBy(
    'COUNTRY_FULL', 'month'
).agg(
    F.mean('WDSP').alias('mean_monthly_WDSP')
).groupBy(
    'COUNTRY_FULL'
).agg(
    F.mean('mean_monthly_WDSP').alias('monthly_WDSP')
).orderBy(
    F.col('monthly_WDSP'), ascending=False
).show()

+--------------------+------------------+
|        COUNTRY_FULL|      monthly_WDSP|
+--------------------+------------------+
|FALKLAND ISLANDS ...|17.853323291643207|
|               ARUBA|15.977568466445604|
|       FAROE ISLANDS|15.004640085348584|
|            BARBADOS|14.104861833330643|
|ST. PIERRE AND MI...|13.964271671947186|
|FRENCH SOUTHERN A...|13.721410918455398|
|          CAPE VERDE|13.689624769963283|
|          MAURITANIA|13.049711560945854|
|     TROMELIN ISLAND|12.996336024862229|
|          ST. HELENA| 12.70270694631401|
|          ANTARCTICA|12.317377902614764|
|             SOMALIA|12.270474730728104|
|COCOS (KEELING) I...| 12.04136264134967|
|            GUERNSEY|12.027518638882889|
|        MAN  ISLE OF|11.902185717540952|
|          MONTSERRAT|11.813225809104981|
|             ICELAND|11.793342469451671|
|      WESTERN SAHARA|11.652829126173785|
|           ST. LUCIA|11.580525361820179|
|            SVALBARD|11.489402906171904|
+--------------------+------------

Q: Which country had the second highest average mean wind speed over the year?

A: ARUBA|15.977568466445604

Makes sense! It's right in the middle of the Caribbean, which has hurricanes.